In [1]:
from dataset import *
import torch

In [7]:
DATA_FILE = 'E:/xplore_data/data/images.h5'
DHSGPS_FILE = 'data/dhs_gps.csv'
data = TestDataset(DATA_FILE, DHSGPS_FILE, normalize=False, K=833)
loader = torch.utils.data.DataLoader(data, batch_size=32, shuffle=False)

In [10]:
def online_mean_and_sd(loader):
    """Compute the mean and sd in an online fashion

        Var[x] = E[X^2] - E^2[X]
    """
    cnt = 0
    fst_moment = torch.empty(9)
    snd_moment = torch.empty(9)

    for data, _, _ in loader:

        b, c, h, w = data.shape
        nb_pixels = b * h * w
        sum_ = torch.sum(data, dim=[0, 2, 3])
        sum_of_square = torch.sum(data ** 2, dim=[0, 2, 3])
        fst_moment = (cnt * fst_moment + sum_) / (cnt + nb_pixels)
        snd_moment = (cnt * snd_moment + sum_of_square) / (cnt + nb_pixels)

        cnt += nb_pixels

    return fst_moment, torch.sqrt(snd_moment - fst_moment ** 2)

In [11]:
stats = online_mean_and_sd(loader)
stats

(tensor([10.1707, 14.8429, 25.7929, 30.0618, 73.5301, 69.8245, 48.3126, 69.7498,
         62.0024]),
 tensor([ 6.0652,  7.4589, 10.7411, 18.1305, 17.0735, 29.9055, 28.0655,  8.7038,
          8.7830]))